## Gestión de ingresos de aerolíneas

¿A qué tarifa se tienen que asignar asientos del avión para maximizar los ingresos?


### Formulación matemática

* $t \in T$: tarifas de asientos
* $s_t$: nº de asientos asignados a cada tarifa 
* $p_t$: precio de un asiento en cada tarifa
* $d_t$: tope de demanda de asientos en cada tarifa
* $S$: asientos del avión

\begin{eqnarray*}
\max_{s_t} & \sum_{t \in T} p_t s_t \\
\text{s.t.} & \\
  & s_t \leq d_t & \forall t \in T \\
  & \sum_{t \in T} s_t \leq S \\
  & s_t \geq 0 & \forall t \in T
\end{eqnarray*}



### Datos

Variables|Tarifa regular|Tarifa con descuento
-|-|-
Precio (eur)|400|150 
Tope de demanda (asientos)|75|125

* $S$ = 150

In [1]:
from pandas import Series

In [2]:
# Nº de asientos
total_seats = 150

# Lista con las tarifas disponibles
tariffs = ["Regular", "Discount"]

print(total_seats, tariffs)

150 ['Regular', 'Discount']


In [3]:
# Serie con los precios de cada tarifa
prices = Series(index=tariffs, data=[400, 150])
prices

Regular     400
Discount    150
dtype: int64

In [4]:
# Serie con la demanda de cada tarifa
demands = Series(index=tariffs, data=[75, 125])
demands

Regular      75
Discount    125
dtype: int64

### Resolución con PuLP

In [5]:
!pip install pulp
from pulp import *

In [27]:
# Crear problema de maximización
prob = LpProblem("Seats", LpMaximize)
prob

Seats:
MAXIMIZE
None
VARIABLES

In [28]:
# Crear variables de asientos de cada tarifa 
seats = LpVariable.dicts("Seats", indices=tariffs, lowBound=0, cat=LpInteger) 
seats

{'Regular': Seats_Regular, 'Discount': Seats_Discount}

In [29]:
# Añadir función objetivo al problema
prob += lpSum([prices[i] * seats[i] for i in tariffs])
prob

Seats:
MAXIMIZE
150*Seats_Discount + 400*Seats_Regular + 0
VARIABLES
0 <= Seats_Discount Integer
0 <= Seats_Regular Integer

In [30]:
# Añadir restricciones de no sobrepasar topes de demandas
for t in tariffs:
    prob += (seats[t] <= demands[t])
prob

Seats:
MAXIMIZE
150*Seats_Discount + 400*Seats_Regular + 0
SUBJECT TO
_C1: Seats_Regular <= 75

_C2: Seats_Discount <= 125

VARIABLES
0 <= Seats_Discount Integer
0 <= Seats_Regular Integer

In [32]:
# Añadir restricción de no sobrepasar total de asientos
prob += (lpSum([seats[i] for i in tariffs]) <= total_seats)

In [33]:
# Comprobar que el problema está bien formulado
prob

Seats:
MAXIMIZE
150*Seats_Discount + 400*Seats_Regular + 0
SUBJECT TO
_C1: Seats_Regular <= 75

_C2: Seats_Discount <= 125

_C3: Seats_Discount + Seats_Regular <= 150

VARIABLES
0 <= Seats_Discount Integer
0 <= Seats_Regular Integer

In [36]:
# Resolver el problema, pintar la solución y ver cuánto da de objetivo
status = prob.solve()
print("Status: ", LpStatus[status])
for v in prob.variables():
  val = value(v)
  if val > 0.0: 
    print(v.name, "=", v.varValue)
round(value(prob.objective), 2)

Status:  Optimal
Seats_Discount = 75.0
Seats_Regular = 75.0


41250.0

* ¿Qué pasaría si en lugar de maximizar la ganancia buscamos minimizarla?
* ¿Y si en las restricciones decimos que la suma de asientos debe ser exactamente $S$, en vez de menor o igual que $S$?
* ¿Y si suponemos que las demandas que nos dan son las mínimas que se deben satisfacer, en lugar de ser las máximas?